In [1]:
# ==== 0. Cargar librerías ====
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.datasets import mnist
from TheModel import build

In [3]:
# ==== 1. Cargar modelos locales ====
import glob
loaded_local_models = [tf.keras.models.load_model(f) for f in glob.glob("mnist_parte_*.keras")]

print(f"{len(loaded_local_models)} modelos locales cargados.")

# Verificar arquitectura
for i in range(len(loaded_local_models)-1):
    assert loaded_local_models[i].summary() == loaded_local_models[i+1].summary(), "¡Los modelos tienen arquitecturas distintas!"

# ==== 2. Cargar test completo ====
(_, _), (x_test, y_test) = mnist.load_data()
x_test = x_test / 255.0
x_test = x_test.reshape(-1, 28, 28)

# ==== 3. Obtener pesos de cada modelo ====
local_weights = [model.get_weights() for model in loaded_local_models]

1 modelos locales cargados.


In [ ]:
# ==== MÉTODO 1: FedAvg ====
# Promedio simple de pesos
fedavg_weights = [np.mean(np.array(w), axis=0) for w in zip(*local_weights)]

# ==== MÉTODO 2: FedMedian ====
# Mediana de pesos por capa
fedmedian_weights = [np.median(np.array(w), axis=0) for w in zip(*local_weights)]

# ==== MÉTODO 3: FedWeightedAvg ====
# Promedio ponderado según tamaño de datos de entrenamiento local
local_sizes = []
for i in range(1, 6):
    d = np.load(f"mnist_parte_{i}.npz")
    local_sizes.append(len(d['labels']))

total_size = sum(local_sizes)
weights_scaled = []
for i, lw in enumerate(local_weights):
    scale = local_sizes[i] / total_size
    weights_scaled.append([w * scale for w in lw])

fedweighted_weights = [np.sum(np.array(ws), axis=0) for ws in zip(*weights_scaled)]

In [ ]:
# ==== 4. Función para evaluar un modelo global ====
def evaluate_global_model(weights, name):
    model = build.build_it()
    model.set_weights(weights)
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    print(f"=== Evaluación: {name} ===")
    print(classification_report(y_test, y_pred_classes))
    model.save(f"{name}.keras")

In [ ]:
# ==== 5. Evaluar cada estrategia de agregación ====
evaluate_global_model(fedavg_weights, "global_model_FedAvg")
evaluate_global_model(fedmedian_weights, "global_model_FedMedian")
evaluate_global_model(fedweighted_weights, "global_model_FedWeightedAvg")

In [ ]:
# ==== 6. Para comparar: modelo centralizado ====
# Cargar datos
x_train_parts = []
y_train_parts = []

for i in range(1, 6):
    d = np.load(f"mnist_parte_{i}.npz")
    x_train_parts.append(d['images'])
    y_train_parts.append(d['labels'])

x_train = np.concatenate(x_train_parts) / 255.0
y_train = np.concatenate(y_train_parts)
x_train = x_train.reshape(-1, 28, 28)

# Entrenamiento central
model_central = build.build_it()
history = model_central.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
model_central.save("global_model_Centralized.keras")

# Evaluación central
y_pred = model_central.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print("=== Evaluación: Modelo Centralizado ===")
print(classification_report(y_test, y_pred_classes))

# Graficar accuracy
plt.plot(history.history['accuracy'], label='Accuracy Centralized')
plt.plot(history.history['val_accuracy'], label='Val Accuracy Centralized')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Centralized Training")
plt.legend()
plt.show()